# Functional model把每一層都表示成一個張量，可以自由排列調用。

### Sequential model則是一個input之後一條路往下走，走到結束，是functional model的簡化特例。

# 一般的 Full connected layer （層的定義：輸入是張量，輸出也是張量）

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

# This returns a tensor，如有一個含有784像素的一維矩陣
inputs = Input(shape=(784,))

# a layer instance is callable on a tensor, and returns a tensor
#層的定義：輸入是張量，輸出也是張量
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x) #做成預測0~9其中一項的預測

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels)  # starts training

# 所有層均可以調用

In [ ]:
x = Input(shape=(784,))
# This works, and returns the 10-way softmax we defined above.
y = model(x)

# 處理時間序列 

In [ ]:
from keras.layers import TimeDistributed

# Input tensor for sequences of 20 timesteps,
# each containing a 784-dimensional vector
input_sequences = Input(shape=(20, 784))

# This applies our previous model to every timestep in the input sequences.
# the output of the previous model was a 10-way softmax,
# so the output of the layer below will be a sequence of 20 vectors of size 10.
processed_sequences = TimeDistributed(model)(input_sequences)  #此處就調用了上面的input的層

# 多輸入和多輸出模型

我們希望預測Twitter上一條新聞會被轉發和點贊多少次。模型的主要輸入是新聞本身，也就是一個詞語的序列。但我們還可以擁有額外的輸入，如新聞發布的日期等。這個模型的損失函數將由兩部分組成，輔助的損失函數評估僅僅基於新聞本身做出預測的情況，主損失函數評估基於新聞和額外信息的預測的情況，即使來自主損失函數的梯度發生彌散，來自輔助損失函數的信息也能夠訓練Embeddding和LSTM層。在模型中早點使用主要的損失函數是對於深度網絡的一個良好的正則方法。

此範本要做語意分析，100是每一個發文的字數，10000是我們字典裡的字庫，512是我們刻意把這些數字投影到該維度上使電腦可以自行分類語意。

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

# Headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# Note that we can name any layer by passing it a "name" argument.
main_input = Input(shape=(100,), dtype='int32', name='main_input')

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# A LSTM will transform the vector sequence into a single vector,
# containing information about the entire sequence
lstm_out = LSTM(32)(x)  #變成一個單元


#我們插入一個額外的損失，使得即使在主損失很高的情況下，LSTM和Embedding層也可以平滑的訓練。
auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)


#再然後，我們將LSTM與額外的輸入數據串聯（concatenate）起來組成輸入，送入模型中：
auxiliary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

# We stack a deep densely-connected network on top
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# And finally we add the main logistic regression layer
main_output = Dense(1, activation='sigmoid', name='main_output')(x)


#最後，我們定義整個2輸入，2輸出的模型：
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])


#模型定義完畢，下一步編譯模型。我們給額外的損失賦0.2的權重。(因為有兩個loss因此要定義好權重)
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
              loss_weights=[1., 0.2])

model.fit([headline_data, additional_data], [labels, labels],
          epochs=50, batch_size=32)

model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

# And trained it via:
model.fit({'main_input': headline_data, 'aux_input': additional_data},
          {'main_output': labels, 'aux_output': labels},
          epochs=50, batch_size=32)



# 共享層: 若要對不同的輸入共享同一層，就初始化該層一次，然後多次調用它

考慮微博數據，我們希望建立模型來判別兩條微博是否是來自同一個用戶，這個需求同樣可以用來判斷一個用戶的兩條微博的相似性。一種實現方式是，我們建立一個模型，它分別將兩條微博的數據映射到兩個特徵向量上，然後將特徵向量串聯並加一個logistic回歸層，輸出它們來自同一個用戶的概率。這種模型的訓練數據是一對對的微博。因為這個問題是對稱的，所以處理第一條微博的模型當然也能重用於處理第二條微博。所以這裡我們使用一個共享的LSTM層來進行映射。

首先，我們將微博的數據轉為（140，256）的矩陣，即每條微博有140個字符，每個單詞的特徵由一個256維的詞向量表示，向量的每個元素為1表示某個字符出現，為0表示不出現，這是一個one-hot編碼。<bt/>

之所以是（140，256）是因為一條微博最多有140個字符，而擴展的ASCII碼表編碼了常見的256個字符。原文中此處為Tweet，所以對外國人而言這是合理的。如果考慮中文字符，那一個單詞的詞向量就不止256了。

In [ ]:
import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

tweet_a = Input(shape=(140, 256))
tweet_b = Input(shape=(140, 256))


#若要對不同的輸入共享同一層，就初始化該層一次，然後多次調用它
# This layer can take as input a matrix
# and will return a vector of size 64
shared_lstm = LSTM(64)

# When we reuse the same layer instance
# multiple times, the weights of the layer
# are also being reused
# (it is effectively *the same* layer)
encoded_a = shared_lstm(tweet_a)
encoded_b = shared_lstm(tweet_b)

# We can then concatenate the two vectors:
merged_vector = keras.layers.concatenate([encoded_a, encoded_b], axis=-1)

# And add a logistic regression on top
predictions = Dense(1, activation='sigmoid')(merged_vector)

# We define a trainable model linking the
# tweet inputs to the predictions
model = Model(inputs=[tweet_a, tweet_b], outputs=predictions)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit([data_a, data_b], labels, epochs=10)

# 層“節點”的概念

無論何時，當你在某個輸入上調用層時，你就創建了一個新的張量（即該層的輸出），同時你也在為這個層增加一個“（計算）節點”。這個節點將輸入張量映射為輸出張量。當你多次調用該層時，這個層就有了多個節點，其下標分別為0，1，2...

在上一版本的Keras中，你可以通過layer.get_output()方法來獲得層的輸出張量，或者通過layer.output_shape獲得其輸出張量的shape。這個版本的Keras你仍然可以這麼做（除了layer.get_output()被output替換）。但如果一個層與多個輸入相連，會出現什麼情況呢？

如果層只與一個輸入相連，那沒有任何困惑的地方。.output將會返回該層唯一的輸出

In [ ]:
a = Input(shape=(140, 256))

lstm = LSTM(32)
encoded_a = lstm(a)

assert lstm.output == encoded_a

但當層與多個輸入相連時，會出現問題: 以下會報錯！！

In [ ]:
a = Input(shape=(140, 256))
b = Input(shape=(140, 256))

lstm = LSTM(32)
encoded_a = lstm(a)
encoded_b = lstm(b)

lstm.output

通過下面這種調用方式即可解決

In [ ]:
assert lstm.get_output_at(0) == encoded_a
assert lstm.get_output_at(1) == encoded_b

Conv2D也一樣

In [ ]:
a = Input(shape=(3, 32, 32)) #3色，32x32像素
b = Input(shape=(3, 64, 64)) #3色，64x64像素

conv = Conv2D(16, (3, 3), padding='same')
conved_a = conv(a)

# Only one input so far, the following will work:
assert conv.input_shape == (None, 3, 32, 32)

conved_b = conv(b)
# now the `.input_shape` property wouldn't work, but this does:
assert conv.get_input_shape_at(0) == (None, 3, 32, 32)
assert conv.get_input_shape_at(1) == (None, 3, 64, 64)

# inception模型

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Input

# input tensor for a 3-channel 256x256 image
input_img = Input(shape=(3, 256, 256))

tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_img)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)

# 卷積層的殘差連接

In [ ]:
from keras.layers import Conv2D, Input

# input tensor for a 3-channel 256x256 image
x = Input(shape=(3, 256, 256))
# 3x3 conv with 3 output channels (same as input channels)
y = Conv2D(3, (3, 3), padding='same')(x)
# this returns x + y.
z = keras.layers.add([x, y])

# 共享視覺模型

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten
from keras.models import Model

# First, define the vision modules
digit_input = Input(shape=(1, 27, 27))
x = Conv2D(64, (3, 3))(digit_input)
x = Conv2D(64, (3, 3))(x)
x = MaxPooling2D((2, 2))(x)
out = Flatten()(x)

vision_model = Model(digit_input, out)

# Then define the tell-digits-apart model
digit_a = Input(shape=(1, 27, 27))
digit_b = Input(shape=(1, 27, 27))

# The vision model will be shared, weights and all
out_a = vision_model(digit_a)
out_b = vision_model(digit_b)

concatenated = keras.layers.concatenate([out_a, out_b])
out = Dense(1, activation='sigmoid')(concatenated)

classification_model = Model([digit_a, digit_b], out)

# 視覺問答模型

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model, Sequential

# First, let's define a vision model using a Sequential model.
# This model will encode an image into a vector.
vision_model = Sequential()
vision_model.add(Conv2D(64, (3, 3) activation='relu', padding='same', input_shape=(3, 224, 224)))
vision_model.add(Conv2D(64, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
vision_model.add(Conv2D(128, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
vision_model.add(Conv2D(256, (3, 3), activation='relu'))
vision_model.add(Conv2D(256, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Flatten())

# Now let's get a tensor with the output of our vision model:
image_input = Input(shape=(3, 224, 224))
encoded_image = vision_model(image_input)

# Next, let's define a language model to encode the question into a vector.
# Each question will be at most 100 word long,
# and we will index words as integers from 1 to 9999.
question_input = Input(shape=(100,), dtype='int32')
embedded_question = Embedding(input_dim=10000, output_dim=256, input_length=100)(question_input)
encoded_question = LSTM(256)(embedded_question)

# Let's concatenate the question vector and the image vector:
merged = keras.layers.concatenate([encoded_question, encoded_image])

# And let's train a logistic regression over 1000 words on top:
output = Dense(1000, activation='softmax')(merged)

# This is our final model:
vqa_model = Model(inputs=[image_input, question_input], outputs=output)

# The next stage would be training this model on actual data.

# 視頻問答模型：圖片變成時間序列，加入LSTM

In [ ]:
from keras.layers import TimeDistributed

video_input = Input(shape=(100, 3, 224, 224))
# This is our video encoded via the previously trained vision_model (weights are reused)
encoded_frame_sequence = TimeDistributed(vision_model)(video_input)  # the output will be a sequence of vectors
encoded_video = LSTM(256)(encoded_frame_sequence)  # the output will be a vector

# This is a model-level representation of the question encoder, reusing the same weights as before:
question_encoder = Model(inputs=question_input, outputs=encoded_question)

# Let's use it to encode the question:
video_question_input = Input(shape=(100,), dtype='int32')
encoded_video_question = question_encoder(video_question_input)

# And this is our video question answering model:
merged = keras.layers.concatenate([encoded_video, encoded_video_question])
output = Dense(1000, activation='softmax')(merged)
video_qa_model = Model(inputs=[video_input, video_question_input], outputs=output)